# VacationPy
----

In [1]:
%%html
<style>
    .jupyter-widgets-disconnected:before { display: none; }
</style>

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv(os.path.join("output_data","cities.csv"))
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(g_key)
map = gmaps.figure(zoom_level=2, center=(20,0))

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Lat", "Lng"]].to_records(index=False)
weight = cities_df["Humidity"].to_list()

# Add Heatmap layer to map
map.add_layer(gmaps.heatmap_layer(locations, weight, point_radius=20))
map

Figure(layout=FigureLayout(height='420px'))

In [5]:
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
narrowed_city_df = cities_df[
    # A max temperature lower than 80 degrees
    (cities_df["Max Temp"] < 80) &
    #  but higher than 70.
    (cities_df["Max Temp"] > 70) &
    # Wind speed less than 10 mph.
    (cities_df["Wind Speed"] < 10) &
    # Zero cloudiness.
    (cities_df["Cloudiness"] == 0)
].copy()

narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame({
    # Add a "Hotel Name" column to the DataFrame
    "Hotel Name": [],
    "Lat": [],
    "Lng": []
})

# Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "types": "lodging",
    "radius": "5000",
    "key": g_key,
}

# Hit the Google Places API for each city's coordinates
for index, row in narrowed_city_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    hotel = requests.get(base_url, params = params).json()
    try:
        # Store the first Hotel result into the DataFrame
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
        narrowed_city_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = hotel["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotel["results"][0]["geometry"]["location"]["lng"]
    except IndexError:
        pass # Welp, guess there's nowhere to stay in this lovely city

hotel_df.head()

,Hotel Name,Lat,Lng
7,ABBA Guesthouse,-18.054138,13.843886
70,Hotel Escarpas do Lago,-20.645701,-46.009734
88,Pond Country Club,-30.167819,-50.255629
138,La Case Bambou,-20.297942,44.269427
139,Dandeli Lake County,15.323247,74.506850


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
gmaps.configure("temp")
# Add marker layer ontop of heat map
map.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))

# Display Map
map

Figure(layout=FigureLayout(height='420px'))